In [13]:
# from web3 import Web3

# # URL do nó da Polygon
# polygon_url = "https://polygon-rpc.com"
# web3 = Web3(Web3.HTTPProvider(polygon_url))

# # Endereço do contrato do BRZ
# brz_contract = web3.eth.contract(
#     address=web3.to_checksum_address("0x4fabb145d64652a948d72533023f6e7a623c7c53"),
#     abi=[
#         {
#             "constant": True,
#             "inputs": [],
#             "name": "decimals",
#             "outputs": [{"name": "", "type": "uint8"}],
#             "type": "function",
#         }
#     ],
# )

# # Consultar os decimais
# decimals = brz_contract.functions.decimals().call()
# print(f"Decimais do BRZ: {decimals}")

In [1]:
import requests

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, src_decimals, dest_decimals, amount):
    params = {
        'srcToken': src_token,  # Token de origem
        'destToken': dest_token,  # Token de destino
        'amount': str(amount * 10**src_decimals),  # Quantidade em unidades menores (wei)
        'network': '137',  # Rede Polygon
        'side': 'SELL',  # Vender token de origem
        'srcDecimals': src_decimals,  # Decimais do token de origem
        'destDecimals': dest_decimals  # Decimais do token de destino
    }

    response = requests.get('https://apiv5.paraswap.io/prices', params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f'Erro: {response.status_code}, {response.json()}'


In [6]:

def parse_swap_data(data):
    price_route = data['priceRoute']
    src_amount = int(price_route['srcAmount']) / (10 ** price_route['srcDecimals'])
    dest_amount = int(price_route['destAmount']) / (10 ** price_route['destDecimals'])
    gas_cost_usd = float(price_route.get('gasCostUSD', 0))
    
    # Taxas e impacto
    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    price_impact_percent = 0.0039  # Impacto de Preço: 0.39%
    
    # Cálculos
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee
    # dest_after_impact = dest_after_fee * (1 - price_impact_percent)
    
    return {
        "Origem (BRZ)": f"{src_amount:.6f}",
        "Destino Bruto (USDT)": f"{dest_amount:.6f}",
        "Taxa de Swap (USDT)": f"{swap_fee:.6f}",
        "Destino Líquido (Após Taxas)": f"{dest_after_fee:.6f}",
        # "Destino Final (Após Impacto de Preço)": f"{dest_after_impact:.6f}",
        "Custo de Gás (USD)": f"${gas_cost_usd:.6f}",
    }


In [7]:


# Consulta BRZ -> USDT
brz_to_usdt = consultar_preco(
    src_token='0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc',  # BRZ
    dest_token='0xc2132d05d31c914a87c6611c10748aeb04b58e8f',  # USDT
    src_decimals=18,
    dest_decimals=6,
    amount=350 # BRZ
)
print("\nBRZ -> USDT:")
print(brz_to_usdt)



BRZ -> USDT:
{'priceRoute': {'blockNumber': 65432123, 'network': 137, 'srcToken': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'srcDecimals': 18, 'srcAmount': '350000000000000000000', 'destToken': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'destDecimals': 6, 'destAmount': '57900780', 'bestRoute': [{'percent': 100, 'swaps': [{'srcToken': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'srcDecimals': 18, 'destToken': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'destDecimals': 6, 'swapExchanges': [{'exchange': 'UniswapV3', 'srcAmount': '350000000000000000000', 'destAmount': '57900780', 'percent': 100, 'poolAddresses': ['0x4af62a93775c093af3e32949ef23c9a323d20817'], 'data': {'path': [{'tokenIn': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'tokenOut': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'fee': '500', 'currentFee': '500'}], 'gasUSD': '0.002111'}}]}]}], 'gasCostUSD': '0.004091', 'gasCost': '211297', 'side': 'SELL', 'version': '5', 'contractAddress': '0xDEF171Fe48CF0115B1d80b8

In [8]:
# Processar e exibir o resumo
resultado = parse_swap_data(brz_to_usdt)
for campo, valor in resultado.items():
    print(f"{campo}: {valor}")

Origem (BRZ): 350.000000
Destino Bruto (USDT): 57.900780
Taxa de Swap (USDT): 0.086851
Destino Líquido (Após Taxas): 57.813929
Custo de Gás (USD): $0.004091


In [17]:
# Consulta USDT -> BRLA
brla_to_usdt = consultar_preco(
    src_token='0xc2132d05d31c914a87c6611c10748aeb04b58e8f',  # USDT
    dest_token='0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc',  # BRLA
    src_decimals=6,
    dest_decimals=18,
    amount=57.81 # USDT
)
print("\n USDT -> BRLA:")
print(brla_to_usdt)


 USDT -> BRLA:
{'priceRoute': {'blockNumber': 65432225, 'network': 137, 'srcToken': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'srcDecimals': 6, 'srcAmount': '57810000', 'destToken': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'destDecimals': 18, 'destAmount': '349054396062950274020', 'bestRoute': [{'percent': 100, 'swaps': [{'srcToken': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'srcDecimals': 6, 'destToken': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'destDecimals': 18, 'swapExchanges': [{'exchange': 'UniswapV3', 'srcAmount': '57810000', 'destAmount': '349054396062950274020', 'percent': 100, 'poolAddresses': ['0x4af62a93775c093af3e32949ef23c9a323d20817'], 'data': {'path': [{'tokenIn': '0xc2132d05d31c914a87c6611c10748aeb04b58e8f', 'tokenOut': '0x4ed141110f6eeeaba9a1df36d8c26f684d2475dc', 'fee': '500', 'currentFee': '500'}], 'gasUSD': '0.002042'}}]}]}], 'gasCostUSD': '0.003959', 'gasCost': '211300', 'side': 'SELL', 'version': '5', 'contractAddress': '0xDEF171Fe48CF0115B1d80

In [18]:
# Processar e exibir o resumo
resultado = parse_swap_data(brla_to_usdt)
for campo, valor in resultado.items():
    print(f"{campo}: {valor}")

Origem (BRZ): 57.810000
Destino Bruto (USDT): 349.054396
Taxa de Swap (USDT): 0.523582
Destino Líquido (Após Taxas): 348.530814
Custo de Gás (USD): $0.003959


In [19]:
import requests

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(amount * 10 ** TOKENS[src_token]["decimals"]),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])
    gas_cost_usd = float(price_route.get("gasCostUSD", 0))

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "Origem": f"{src_amount:.6f} {src_token}",
        "Destino Bruto": f"{dest_amount:.6f} {dest_token}",
        "Taxa de Swap": f"{swap_fee:.6f} {dest_token}",
        "Destino Líquido (Após Taxas)": f"{dest_after_fee:.6f} {dest_token}",
        "Custo de Gás (USD)": f"${gas_cost_usd:.6f}",
    }

# Função para exibir resultados
def exibir_resultado(resultado):
    if "error" in resultado:
        print(f"Erro: {resultado['error']}")
        if "details" in resultado:
            print(f"Detalhes: {resultado['details']}")
    else:
        for campo, valor in resultado.items():
            print(f"{campo}: {valor}")
    print("-" * 50)



In [20]:
# Consultas de exemplo
print("BRZ -> USDT")
brz_to_usdt_data = consultar_preco("BRZ", "USDT", 350)
brz_to_usdt_result = parse_swap_data(brz_to_usdt_data, "BRZ", "USDT")
exibir_resultado(brz_to_usdt_result)

print("USDT -> BRLA")
usdt_to_brla_data = consultar_preco("USDT", "BRLA", 57.81)
usdt_to_brla_result = parse_swap_data(usdt_to_brla_data, "USDT", "BRLA")
exibir_resultado(usdt_to_brla_result)

BRZ -> USDT
Origem: 350.000000 BRZ
Destino Bruto: 57.900780 USDT
Taxa de Swap: 0.086851 USDT
Destino Líquido (Após Taxas): 57.813929 USDT
Custo de Gás (USD): $0.004090
--------------------------------------------------
USDT -> BRLA
Origem: 57.810000 USDT
Destino Bruto: 349.054396 BRLA
Taxa de Swap: 0.523582 BRLA
Destino Líquido (Após Taxas): 348.530814 BRLA
Custo de Gás (USD): $0.004093
--------------------------------------------------


In [ ]:
import requests
import time

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

# Função para executar a estratégia triangular
def executar_estrategia(brz_amount, route):
    if route == "USDT":
        # Etapa 1: BRZ -> USDT
        brz_to_usdt_data = consultar_preco("BRZ", "USDT", brz_amount)
        if "error" in brz_to_usdt_data:
            return {"error": "Erro na conversão BRZ -> USDT", "details": brz_to_usdt_data}

        brz_to_usdt_result = parse_swap_data(brz_to_usdt_data, "BRZ", "USDT")
        usdt_received = brz_to_usdt_result["dest_amount_after_fee"]

        # Arredondar USDT para 2 casas decimais
        usdt_received = int(usdt_received * 100) / 100.0

        # Etapa 2: USDT -> BRLA
        usdt_to_brla_data = consultar_preco("USDT", "BRLA", usdt_received)
        if "error" in usdt_to_brla_data:
            return {"error": "Erro na conversão USDT -> BRLA", "details": usdt_to_brla_data}

        usdt_to_brla_result = parse_swap_data(usdt_to_brla_data, "USDT", "BRLA")
        brla_received = usdt_to_brla_result["dest_amount_after_fee"]

    elif route == "POL":
        # Etapa 1: BRZ -> POL
        brz_to_pol_data = consultar_preco("BRZ", "POL", brz_amount)
        if "error" in brz_to_pol_data:
            return {"error": "Erro na conversão BRZ -> POL", "details": brz_to_pol_data}

        brz_to_pol_result = parse_swap_data(brz_to_pol_data, "BRZ", "POL")
        pol_received = brz_to_pol_result["dest_amount_after_fee"]

        # Arredondar POL para 2 casas decimais
        pol_received = int(pol_received * 100) / 100.0

        # Etapa 2: POL -> BRLA
        pol_to_brla_data = consultar_preco("POL", "BRLA", pol_received)
        if "error" in pol_to_brla_data:
            return {"error": "Erro na conversão POL -> BRLA", "details": pol_to_brla_data}

        pol_to_brla_result = parse_swap_data(pol_to_brla_data, "POL", "BRLA")
        brla_received = pol_to_brla_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo
    lucro = brla_received - brz_amount

    return {
        "Rota": route,
        "BRZ inicial": brz_amount,
        "BRLA final": brla_received,
        "Lucro/Prejuízo": lucro,
    }

# Loop para execução contínua
def iniciar_estrategia_em_loop(brz_amount, intervalo=10):
    ultimo_resultado_usdt = None  # Armazena o último resultado para a rota USDT
    ultimo_resultado_pol = None  # Armazena o último resultado para a rota POL

    while True:
        try:
            # Estratégia com rota USDT
            resultado_usdt = executar_estrategia(brz_amount, "USDT")
            if resultado_usdt != ultimo_resultado_usdt:
                print("Estratégia com USDT Executada:")
                for chave, valor in resultado_usdt.items():
                    if isinstance(valor, float):
                        print(f"{chave}: {valor:.6f}")
                    else:
                        print(f"{chave}: {valor}")
                print("-" * 50)
                ultimo_resultado_usdt = resultado_usdt

            # Estratégia com rota POL
            resultado_pol = executar_estrategia(brz_amount, "POL")
            if resultado_pol != ultimo_resultado_pol:
                print("Estratégia com POL Executada:")
                for chave, valor in resultado_pol.items():
                    if isinstance(valor, float):
                        print(f"{chave}: {valor:.6f}")
                    else:
                        print(f"{chave}: {valor}")
                print("-" * 50)
                ultimo_resultado_pol = resultado_pol

        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)  # Esperar antes de repetir




In [ ]:
# Executar o loop
iniciar_estrategia_em_loop(brz_amount=350, intervalo=10)

In [57]:
import requests
import time

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
    "DAI": {"address": "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063", "decimals": 18},
    "RENDER": {"address": "0x61299774020da444af134c82fa83e3810b309991", "decimals": 18},
    "LINK": {"address": "0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39", "decimals": 18},
    
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

# Função para executar a estratégia triangular
def executar_estrategia_triangular(brla_amount):
    # Etapa 1: BRLA -> USDT
    brla_to_usdt_data = consultar_preco("BRLA", "USDT", brla_amount)
    if "error" in brla_to_usdt_data:
        return {"error": "Erro na conversão BRLA -> USDT", "details": brla_to_usdt_data}

    brla_to_usdt_result = parse_swap_data(brla_to_usdt_data, "BRLA", "USDT")
    usdt_received = brla_to_usdt_result["dest_amount_after_fee"]

    # Etapa 2: USDT -> POL
    usdt_to_pol_data = consultar_preco("USDT", "POL", usdt_received)
    if "error" in usdt_to_pol_data:
        return {"error": "Erro na conversão USDT -> POL", "details": usdt_to_pol_data}

    usdt_to_pol_result = parse_swap_data(usdt_to_pol_data, "USDT", "POL")
    pol_received = usdt_to_pol_result["dest_amount_after_fee"]

    # Etapa 3: POL -> BRLA
    pol_to_brla_data = consultar_preco("POL", "BRLA", pol_received)
    if "error" in pol_to_brla_data:
        return {"error": "Erro na conversão POL -> BRLA", "details": pol_to_brla_data}

    pol_to_brla_result = parse_swap_data(pol_to_brla_data, "POL", "BRLA")
    brla_final = pol_to_brla_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo
    lucro = brla_final - brla_amount

    return {
        "BRLA inicial": brla_amount,
        "USDT recebido": usdt_received,
        "POL recebido": pol_received,
        "BRLA final": brla_final,
        "Lucro/Prejuízo": lucro,
    }

# Loop para execução contínua
def iniciar_estrategia_triangular_em_loop(brla_amount, intervalo=10):
    ultimo_resultado = None  # Armazena o último resultado

    while True:
        try:
            resultado = executar_estrategia_triangular(brla_amount)
            if resultado != ultimo_resultado:
                print("Estratégia Triangular Executada:")
                for chave, valor in resultado.items():
                    if isinstance(valor, float):
                        print(f"{chave}: {valor:.6f}")
                    else:
                        print(f"{chave}: {valor}")
                print("-" * 50)
                ultimo_resultado = resultado
        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)  # Esperar antes de repetir



In [59]:
# Executar o loop
iniciar_estrategia_triangular_em_loop(brla_amount=50, intervalo=1)


Estratégia Triangular Executada:
BRLA inicial: 50
USDT recebido: 8.225922
POL recebido: 13.216935
BRLA final: 49.688057
Lucro/Prejuízo: -0.311943
--------------------------------------------------
Estratégia Triangular Executada:
BRLA inicial: 50
USDT recebido: 8.225922
POL recebido: 13.216935
BRLA final: 49.696405
Lucro/Prejuízo: -0.303595
--------------------------------------------------
Estratégia Triangular Executada:
BRLA inicial: 50
USDT recebido: 8.225922
POL recebido: 13.223855
BRLA final: 49.759642
Lucro/Prejuízo: -0.240358
--------------------------------------------------
Estratégia Triangular Executada:
BRLA inicial: 50
USDT recebido: 8.225922
POL recebido: 13.221339
BRLA final: 49.759024
Lucro/Prejuízo: -0.240976
--------------------------------------------------
Estratégia Triangular Executada:
BRLA inicial: 50
USDT recebido: 8.225922
POL recebido: 13.221339
BRLA final: 49.768996
Lucro/Prejuízo: -0.231004
--------------------------------------------------
Estratégia Tria

KeyboardInterrupt: 

In [66]:
import requests
import time

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
    "DAI": {"address": "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063", "decimals": 18},
    "RENDER": {"address": "0x61299774020da444af134c82fa83e3810b309991", "decimals": 18},
    "LINK": {"address": "0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39", "decimals": 18},
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

# Função para executar a estratégia triangular
def executar_estrategia_triangular(brla_amount, token):
    # Etapa 1: BRLA -> USDT
    brla_to_usdt_data = consultar_preco("BRLA", "USDT", brla_amount)
    if "error" in brla_to_usdt_data:
        return {"error": "Erro na conversão BRLA -> USDT", "details": brla_to_usdt_data}

    brla_to_usdt_result = parse_swap_data(brla_to_usdt_data, "BRLA", "USDT")
    usdt_received = brla_to_usdt_result["dest_amount_after_fee"]

    # Etapa 2: USDT -> TOKEN
    usdt_to_token_data = consultar_preco("USDT", token, usdt_received)
    if "error" in usdt_to_token_data:
        return {"error": f"Erro na conversão USDT -> {token}", "details": usdt_to_token_data}

    usdt_to_token_result = parse_swap_data(usdt_to_token_data, "USDT", token)
    token_received = usdt_to_token_result["dest_amount_after_fee"]

    # Etapa 3: TOKEN -> BRLA
    token_to_brla_data = consultar_preco(token, "BRLA", token_received)
    if "error" in token_to_brla_data:
        return {"error": f"Erro na conversão {token} -> BRLA", "details": token_to_brla_data}

    token_to_brla_result = parse_swap_data(token_to_brla_data, token, "BRLA")
    brla_final = token_to_brla_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo
    lucro = brla_final - brla_amount

    return {
        "TOKEN": token,
        "BRLA inicial": brla_amount,
        "USDT recebido": usdt_received,
        "TOKEN recebido": token_received,
        "BRLA final": brla_final,
        "Lucro/Prejuízo": lucro,
    }

# Loop para execução contínua
def iniciar_estrategia_triangular_em_loop(brla_amount, limiar_lucro, intervalo=10):
    ultimo_resultado = {}  # Armazena o último resultado por token

    while True:
        try:
            for token in TOKENS:
                if token in ["BRLA", "USDT"]:  # Ignorar BRLA e USDT como token intermediário
                    continue

                resultado = executar_estrategia_triangular(brla_amount, token)

                if "error" in resultado:
                    print(f"Erro com {token}: {resultado['details']}")
                    continue

                lucro = resultado["Lucro/Prejuízo"]

                # Imprimir apenas se o lucro for maior que o limiar e diferente do último resultado
                if lucro > limiar_lucro:# and resultado != ultimo_resultado.get(token):
                    print(f"Estratégia Triangular Executada com {token}:")
                    for chave, valor in resultado.items():
                        if isinstance(valor, float):
                            print(f"{chave}: {valor:.6f}")
                        else:
                            print(f"{chave}: {valor}")
                    print("-" * 50)

                ultimo_resultado[token] = resultado

        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)


In [67]:

# Executar o loop com BRLA inicial e limiar de lucro
iniciar_estrategia_triangular_em_loop(brla_amount=50, limiar_lucro=-0.01, intervalo=0.1)


Estratégia Triangular Executada com DAI:
TOKEN: DAI
BRLA inicial: 50
USDT recebido: 10.548690
TOKEN recebido: 10.523913
BRLA final: 63.783081
Lucro/Prejuízo: 13.783081
--------------------------------------------------
Estratégia Triangular Executada com RENDER:
TOKEN: RENDER
BRLA inicial: 50
USDT recebido: 10.548690
TOKEN recebido: 1.137496
BRLA final: 63.520517
Lucro/Prejuízo: 13.520517
--------------------------------------------------


KeyboardInterrupt: 

In [86]:
import requests
import time

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
    "DAI": {"address": "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063", "decimals": 18},
    "RENDER": {"address": "0x61299774020da444af134c82fa83e3810b309991", "decimals": 18},
    "LINK": {"address": "0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39", "decimals": 18},
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

# Função para executar a estratégia BRZ -> TOKEN -> BRLA
def executar_estrategia_brz_to_brla(brz_amount, token):
    # Etapa 1: BRZ -> TOKEN
    brz_to_token_data = consultar_preco("BRZ", token, brz_amount)
    if "error" in brz_to_token_data:
        return {"error": f"Erro na conversão BRZ -> {token}", "details": brz_to_token_data}

    brz_to_token_result = parse_swap_data(brz_to_token_data, "BRZ", token)
    token_received = brz_to_token_result["dest_amount_after_fee"]

    # Etapa 2: TOKEN -> BRLA
    token_to_brla_data = consultar_preco(token, "BRLA", token_received)
    if "error" in token_to_brla_data:
        return {"error": f"Erro na conversão {token} -> BRLA", "details": token_to_brla_data}

    token_to_brla_result = parse_swap_data(token_to_brla_data, token, "BRLA")
    brla_final = token_to_brla_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo
    lucro = brla_final - brz_amount

    return {
        "TOKEN": token,
        "BRZ inicial": brz_amount,
        "TOKEN recebido": token_received,
        "BRLA final": brla_final,
        "Lucro/Prejuízo": lucro,
    }


# Loop para execução contínua
def iniciar_estrategia_brz_to_brla_em_loop(brz_amounts, limiar_lucro, intervalo=10):
    ultimo_resultado = {}  # Armazena o último resultado por token
    count = 0
    while True:
        try:
            for token in TOKENS:
                if token in ["BRZ", "BRLA"]:  # Ignorar BRZ e BRLA como token intermediário
                    continue

                for brz_amount in brz_amounts:
                    resultado = executar_estrategia_brz_to_brla(brz_amount, token)
    
                    if "error" in resultado:
                        print(f"Erro com {token}: {resultado['details']}")
                        continue
    
                    lucro = resultado["Lucro/Prejuízo"]
    
                    # Imprimir apenas se o lucro for maior que o limiar e diferente do último resultado
                    if lucro > limiar_lucro:# and resultado != ultimo_resultado.get(token):
                        count += 1
                        print(f"{count} - Estratégia BRZ -> {token} -> BRLA Executada. Valor: {brz_amount}")
                        for chave, valor in resultado.items():
                            if isinstance(valor, float):
                                print(f"{chave}: {valor:.6f}")
                            else:
                                print(f"{chave}: {valor}")
                        print("-" * 50)
    
                    ultimo_resultado[token] = resultado

        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)



In [87]:

# Executar o loop com BRZ inicial e limiar de lucro
iniciar_estrategia_brz_to_brla_em_loop(brz_amounts=[200], limiar_lucro=0, intervalo=1)

1 - Estratégia BRZ -> USDT -> BRLA Executada. Valor: 200
TOKEN: USDT
BRZ inicial: 200
TOKEN recebido: 33.029825
BRLA final: 200.221413
Lucro/Prejuízo: 0.221413
--------------------------------------------------
2 - Estratégia BRZ -> POL -> BRLA Executada. Valor: 200
TOKEN: POL
BRZ inicial: 200
TOKEN recebido: 53.374613
BRLA final: 200.118950
Lucro/Prejuízo: 0.118950
--------------------------------------------------
3 - Estratégia BRZ -> DAI -> BRLA Executada. Valor: 200
TOKEN: DAI
BRZ inicial: 200
TOKEN recebido: 33.025991
BRLA final: 200.276243
Lucro/Prejuízo: 0.276243
--------------------------------------------------
4 - Estratégia BRZ -> USDT -> BRLA Executada. Valor: 200
TOKEN: USDT
BRZ inicial: 200
TOKEN recebido: 33.029825
BRLA final: 200.221413
Lucro/Prejuízo: 0.221413
--------------------------------------------------
5 - Estratégia BRZ -> POL -> BRLA Executada. Valor: 200
TOKEN: POL
BRZ inicial: 200
TOKEN recebido: 53.383003
BRLA final: 200.115342
Lucro/Prejuízo: 0.115342
--

KeyboardInterrupt: 

In [103]:
import requests
import time

# Dados dos Tokens (endereços e decimais)
TOKENS = {
    # "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    # "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
    "DAI": {"address": "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063", "decimals": 18},
    "RENDER": {"address": "0x61299774020da444af134c82fa83e3810b309991", "decimals": 18},
    "LINK": {"address": "0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39", "decimals": 18},
}

# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}

# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

# Função para executar a estratégia triangular
def executar_estrategia_triangular_blockchain(entrada, token1, token2, valor_inicial):
    # Etapa 1: ENTRADA -> TOKEN1
    entrada_to_token1_data = consultar_preco(entrada, token1, valor_inicial)
    if "error" in entrada_to_token1_data:
        return {"error": f"Erro na conversão {entrada} -> {token1}", "details": entrada_to_token1_data}

    entrada_to_token1_result = parse_swap_data(entrada_to_token1_data, entrada, token1)
    token1_received = entrada_to_token1_result["dest_amount_after_fee"]

    # Etapa 2: TOKEN1 -> TOKEN2
    token1_to_token2_data = consultar_preco(token1, token2, token1_received)
    if "error" in token1_to_token2_data:
        return {"error": f"Erro na conversão {token1} -> {token2}", "details": token1_to_token2_data}

    token1_to_token2_result = parse_swap_data(token1_to_token2_data, token1, token2)
    token2_received = token1_to_token2_result["dest_amount_after_fee"]

    # Etapa 3: TOKEN2 -> ENTRADA
    token2_to_entrada_data = consultar_preco(token2, entrada, token2_received)
    if "error" in token2_to_entrada_data:
        return {"error": f"Erro na conversão {token2} -> {entrada}", "details": token2_to_entrada_data}

    token2_to_entrada_result = parse_swap_data(token2_to_entrada_data, token2, entrada)
    entrada_final = token2_to_entrada_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo
    lucro = entrada_final - valor_inicial

    return {
        "ENTRADA": entrada,
        "TOKEN1": token1,
        "TOKEN2": token2,
        "ENTRADA inicial": valor_inicial,
        "TOKEN1 recebido": token1_received,
        "TOKEN2 recebido": token2_received,
        "ENTRADA final": entrada_final,
        "Lucro/Prejuízo": lucro,
    }

# Loop para execução contínua
def iniciar_estrategia_triangular_blockchain_em_loop(entradas, valor_inicial, limiar_lucro, intervalo=10):
    ultimo_resultado = {}  # Armazena o último resultado por combinação

    while True:
        try:
            for entrada in entradas:
                for token1 in TOKENS:
                    if token1 in [entrada]:  # Ignorar o token de entrada como intermediário
                        continue
                    for token2 in TOKENS:
                        if token2 in [entrada, token1]:  # Ignorar duplicatas
                            continue

                        resultado = executar_estrategia_triangular_blockchain(entrada, token1, token2, valor_inicial)

                        if "error" in resultado:
                            print(f"Erro com {entrada} -> {token1} -> {token2}: {resultado['details']}")
                            continue

                        lucro = resultado["Lucro/Prejuízo"]

                        # Imprimir apenas se o lucro for maior que o limiar e diferente do último resultado
                        chave_resultado = f"{entrada}->{token1}->{token2}"
                        if lucro > limiar_lucro and resultado != ultimo_resultado.get(chave_resultado):
                            print(f"Estratégia Triangular {entrada} -> {token1} -> {token2} -> {entrada} Executada:")
                            for chave, valor in resultado.items():
                                if isinstance(valor, float):
                                    print(f"{chave}: {valor:.6f}")
                                else:
                                    print(f"{chave}: {valor}")
                            print("-" * 50)

                        ultimo_resultado[chave_resultado] = resultado

        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)




In [105]:
# Lista de entradas para testar
entradas = ["USDT", "DAI", "POL", "RENDER", "LINK"]

# Executar o loop com entradas iniciais, valor inicial e limiar de lucro
iniciar_estrategia_triangular_blockchain_em_loop(entradas=entradas, valor_inicial=10000, limiar_lucro=-10, intervalo=1)

Erro com LINK -> USDT -> DAI: {'error': 'Erro: 400', 'details': {'error': 'ESTIMATED_LOSS_GREATER_THAN_MAX_IMPACT', 'value': '21.54%', 'priceRoute': {'blockNumber': 65443841, 'network': 137, 'srcToken': '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063', 'srcDecimals': 18, 'srcAmount': '273453939148236926222336', 'destToken': '0x53e0bca35ec356bd5dddfebbd1fc0fd03fabad39', 'destDecimals': 18, 'destAmount': '7689013944531073842538', 'bestRoute': [{'percent': 16, 'swaps': [{'srcToken': '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063', 'srcDecimals': 18, 'destToken': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174', 'destDecimals': 6, 'swapExchanges': [{'exchange': 'UniswapV3', 'srcAmount': '27345393914823692622234', 'destAmount': '27346635648', 'percent': 62.5, 'poolAddresses': ['0x5645dcb64c059aa11212707fbf4e7f984440a8cf'], 'data': {'path': [{'tokenIn': '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063', 'tokenOut': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174', 'fee': '100', 'currentFee': '100'}], 'gasUSD': 

KeyboardInterrupt: 